# Standalone quad-precision LP solver interface

In [1]:
from cobra.io import load_json_model
from qminospy.solver import QMINOS
import numpy as np

solver = QMINOS()

### Usually, can use  default solver options and just set
`precision = 'quad'`
(or 'dq' or 'dqq')
for quad precision, and
`precision = 'double'`
for double precision

In [2]:
#==================================================
# Runtime options that stay constant for Double and Quad (or DQQ) are set here
#==================================================
# Double for all runs
solver.opt_intdict['lp_d']['Expand frequency'] = 100000
solver.opt_intdict['lp_d']['Scale option'] = 2
solver.opt_realdict['lp_d']['Feasibility tol'] = 1e-7
solver.opt_realdict['lp_d']['Optimality tol'] = 1e-7
solver.opt_realdict['lp_d']['LU factor tol'] = 1.9
solver.opt_realdict['lp_d']['LU update tol'] = 1.9

# Quad of DQQ
solver.opt_intdict['lp']['Expand frequency'] = 100000
solver.opt_intdict['lp']['Scale option'] = 2
solver.opt_realdict['lp']['Feasibility tol'] = 1e-15
solver.opt_realdict['lp']['Optimality tol'] = 1e-15
solver.opt_realdict['lp']['LU factor tol'] = 10.0
solver.opt_realdict['lp']['LU update tol'] = 10.0

In [3]:
#==================================================
# Load model and extract matrices
#==================================================
import os
filei = os.path.join('models','iJO1366.json')

mdl = load_json_model(filei)
mdl = mdl.to_array_based_model()
A = mdl.S
b = mdl.b
c = mdl.objective_coefficients
xl = mdl.lower_bounds
xu = mdl.upper_bounds
csense= mdl.constraint_sense

#==================================================
# Solve
#==================================================
# Quad solution
print('Solving with Quad')
xq,statq,hsq = solver.solvelp(A,b,c,xl,xu,csense,precision='quad')
nRxns = len(c)
fq = np.dot(c, xq[0:nRxns])

# Double solution
print('Solving with Double')
xd,statd,hsd = solver.solvelp(A,b,c,xl,xu,csense,precision='double')
fd = np.dot(c, xd[0:nRxns])

# What was the objective?
obj_rxn = str(mdl.objective)
max_abs_c = max(abs(mdl.objective_coefficients))

Solving with Quad
Getting MINOS parameters...
Done in 0.820994 seconds with status 0
Solving with Double
Getting MINOS parameters...
Done in 0.115483 seconds with status 0


# What was the status of the solver?

In [4]:
print('Quad status:', statq)
print('Double status:', statd)

('Quad status:', array(0))
('Double status:', array(0))


# By how much did objectives differ between DQQ and D?

In [5]:
df = fq - fd
print('Max objective difference between Quad and Double: %g'%df)

Max objective difference between Quad and Double: 2.28928e-13
